# Pandas Essentials: Practical Best Practices

A comprehensive guide to the most commonly used pandas operations for data manipulation and analysis.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## 1. Creating Sample Data

In [ ]:
# Create sample datasets
np.random.seed(42)

# Main dataset
df = pd.DataFrame({
    'id': range(1, 101),
    'name': [f'User_{i}' for i in range(1, 101)],
    'age': np.random.randint(18, 65, 100),
    'salary': np.random.randint(30000, 120000, 100),
    'department': np.random.choice(['IT', 'HR', 'Finance', 'Marketing'], 100),
    'join_date': pd.date_range('2020-01-01', periods=100, freq='3D'),
    'score': np.random.uniform(0, 100, 100)
})

# Add some NaN values
df.loc[np.random.choice(df.index, 10), 'salary'] = np.nan
df.loc[np.random.choice(df.index, 5), 'score'] = np.nan

# Department info dataset for joining
dept_info = pd.DataFrame({
    'department': ['IT', 'HR', 'Finance', 'Marketing', 'Operations'],
    'budget': [500000, 200000, 800000, 300000, 400000],
    'location': ['Building A', 'Building B', 'Building C', 'Building A', 'Building B']
})

print(f"Main dataset shape: {df.shape}")
df.head()

## 2. Selecting Rows and Columns

In [ ]:
# Select single column
names = df['name']

# Select multiple columns
basic_info = df[['name', 'age', 'department']]

# Select columns by position
first_three_cols = df.iloc[:, :3]

# Select rows by position
first_five_rows = df.iloc[:5]

# Select specific rows and columns
subset = df.iloc[10:15, [0, 1, 3]]  # rows 10-14, columns 0,1,3

# Select by label
label_subset = df.loc[10:14, ['name', 'age', 'salary']]

print("Basic info (first 3 rows):")
print(basic_info.head(3))

## 3. Boolean Indexing

In [ ]:
# Single condition
high_earners = df[df['salary'] > 80000]

# Multiple conditions with &, |
young_high_earners = df[(df['age'] < 30) & (df['salary'] > 70000)]

# Using isin() for multiple values
tech_finance = df[df['department'].isin(['IT', 'Finance'])]

# String operations
users_with_5 = df[df['name'].str.contains('5')]

# Null value filtering
complete_salary_data = df[df['salary'].notna()]

# Query method (alternative syntax)
query_result = df.query('age > 40 and department == "IT"')

print(f"High earners: {len(high_earners)} people")
print(f"Young high earners: {len(young_high_earners)} people")
print(f"Tech/Finance: {len(tech_finance)} people")

## 4. Joining and Merging

In [ ]:
# Inner join (default)
df_with_dept = df.merge(dept_info, on='department')

# Left join (keep all rows from left DataFrame)
df_left = df.merge(dept_info, on='department', how='left')

# Right join
df_right = df.merge(dept_info, on='department', how='right')

# Outer join (keep all rows from both)
df_outer = df.merge(dept_info, on='department', how='outer')

# Join on different column names
# df.merge(other_df, left_on='col1', right_on='col2')

# Join on index
# df.merge(other_df, left_index=True, right_index=True)

print(f"Original df: {df.shape}")
print(f"After inner join: {df_with_dept.shape}")
print(f"After left join: {df_left.shape}")
print(f"After outer join: {df_outer.shape}")

df_with_dept.head(3)

## 5. Handling Duplicates

In [ ]:
# Create some duplicates for demonstration
df_with_dupes = pd.concat([df, df.iloc[:5]], ignore_index=True)

# Check for duplicates
print(f"Total rows: {len(df_with_dupes)}")
print(f"Duplicate rows: {df_with_dupes.duplicated().sum()}")

# Remove duplicates (keep first occurrence)
df_no_dupes = df_with_dupes.drop_duplicates()

# Remove duplicates based on specific columns
df_unique_names = df_with_dupes.drop_duplicates(subset=['name'])

# Keep last occurrence instead of first
df_keep_last = df_with_dupes.drop_duplicates(keep='last')

# Find duplicate rows
duplicate_rows = df_with_dupes[df_with_dupes.duplicated(keep=False)]

print(f"After removing duplicates: {len(df_no_dupes)}")
print(f"Unique names only: {len(df_unique_names)}")

## 6. DateTime Operations

In [ ]:
# Convert string to datetime
# df['date_col'] = pd.to_datetime(df['date_col'])

# Extract date components
df['join_year'] = df['join_date'].dt.year
df['join_month'] = df['join_date'].dt.month
df['join_weekday'] = df['join_date'].dt.day_name()

# Calculate time differences
df['days_since_join'] = (pd.Timestamp.now() - df['join_date']).dt.days

# Filter by date range
recent_joins = df[df['join_date'] >= '2020-06-01']

# Resample by time period (requires datetime index)
df_indexed = df.set_index('join_date')
monthly_hires = df_indexed.resample('M').size()

# Date arithmetic
df['review_date'] = df['join_date'] + pd.DateOffset(months=6)

print("Join date analysis:")
print(df[['name', 'join_date', 'join_year', 'join_weekday', 'days_since_join']].head())

print("\nMonthly hiring pattern:")
print(monthly_hires.head())

## 7. Handling Missing Values

In [ ]:
# Check for missing values
print("Missing values per column:")
print(df.isnull().sum())

# Fill missing values with a constant
df_filled_constant = df.fillna(0)

# Fill with column mean/median
df_filled_mean = df.copy()
df_filled_mean['salary'] = df_filled_mean['salary'].fillna(df_filled_mean['salary'].mean())
df_filled_mean['score'] = df_filled_mean['score'].fillna(df_filled_mean['score'].median())

# Forward fill (use previous value)
df_ffill = df.fillna(method='ffill')

# Backward fill
df_bfill = df.fillna(method='bfill')

# Fill with different values per column
fill_values = {'salary': df['salary'].median(), 'score': 0}
df_custom_fill = df.fillna(value=fill_values)

# Drop rows with any missing values
df_no_na = df.dropna()

# Drop rows with missing values in specific columns
df_salary_complete = df.dropna(subset=['salary'])

print(f"\nOriginal shape: {df.shape}")
print(f"After dropping NAs: {df_no_na.shape}")
print(f"After dropping salary NAs: {df_salary_complete.shape}")

## 8. Sampling Data

In [ ]:
# Random sample of rows
sample_10 = df.sample(n=10, random_state=42)

# Sample by percentage
sample_20_percent = df.sample(frac=0.2, random_state=42)

# Sample with replacement
sample_with_replacement = df.sample(n=50, replace=True, random_state=42)

# Stratified sampling by group
stratified_sample = df.groupby('department').apply(lambda x: x.sample(n=min(5, len(x)), random_state=42))
stratified_sample = stratified_sample.reset_index(drop=True)

# Sample top/bottom n rows
top_earners = df.nlargest(10, 'salary')
bottom_earners = df.nsmallest(10, 'salary')

print(f"Random sample: {sample_10.shape}")
print(f"20% sample: {sample_20_percent.shape}")
print(f"Stratified sample: {stratified_sample.shape}")

print("\nTop 5 earners:")
print(top_earners[['name', 'salary', 'department']].head())

## 9. Grouping and Aggregation

In [ ]:
# Basic groupby operations
dept_stats = df.groupby('department').agg({
    'salary': ['mean', 'median', 'count'],
    'age': 'mean',
    'score': 'mean'
})

# Flatten column names
dept_stats.columns = ['_'.join(col).strip() for col in dept_stats.columns]
dept_stats = dept_stats.reset_index()

# Multiple groupby columns
age_dept_stats = df.groupby(['department', pd.cut(df['age'], bins=[0, 30, 50, 100], labels=['Young', 'Middle', 'Senior'])])['salary'].mean()

# Apply custom function
def salary_range(series):
    return series.max() - series.min()

salary_ranges = df.groupby('department')['salary'].apply(salary_range)

print("Department statistics:")
print(dept_stats)

print("\nSalary ranges by department:")
print(salary_ranges)

In [ ]:
age_dept_stats.head()

## 10. Data Transformation

In [ ]:
# Create new columns
df['salary_category'] = pd.cut(df['salary'], bins=[0, 50000, 80000, float('inf')], 
                              labels=['Low', 'Medium', 'High'])

# Apply function to column
df['name_length'] = df['name'].apply(len)

# Map values
dept_mapping = {'IT': 'Technology', 'HR': 'Human Resources', 'Finance': 'Finance', 'Marketing': 'Marketing'}
df['dept_full_name'] = df['department'].map(dept_mapping)

# Conditional logic with np.where
df['senior_employee'] = np.where(df['age'] >= 40, 'Senior', 'Junior')

# Multiple conditions with np.select
conditions = [
    df['age'] < 30,
    (df['age'] >= 30) & (df['age'] < 50),
    df['age'] >= 50
]
choices = ['Young', 'Middle-aged', 'Senior']
df['age_group'] = np.select(conditions, choices, default='Unknown')

# Rank data
df['salary_rank'] = df['salary'].rank(ascending=False)

print("Transformed data sample:")
print(df[['name', 'age', 'salary', 'salary_category', 'age_group', 'salary_rank']].head())

In [ ]:
df.head()

## 11. Pivot Tables and Reshaping

In [ ]:
# Create pivot table
pivot_table = df.pivot_table(
    values='salary', 
    index='department', 
    columns='age_group', 
    aggfunc='mean',
    fill_value=0
)

# Cross-tabulation
crosstab = pd.crosstab(df['department'], df['age_group'], margins=True)

# Melt (wide to long format)
df_subset = df[['name', 'age', 'salary', 'score']].head(5)
melted = df_subset.melt(id_vars=['name'], value_vars=['age', 'salary', 'score'])

print("Pivot table - Average salary by department and age group:")
print(pivot_table)

print("\nCross-tabulation:")
print(crosstab)

In [ ]:
melted.head()

## 12. String Operations

In [ ]:
# String methods
df['name_upper'] = df['name'].str.upper()
df['name_lower'] = df['name'].str.lower()

# Extract parts of strings
df['user_number'] = df['name'].str.extract(r'User_(\d+)').astype(int)

# String contains
contains_1 = df[df['name'].str.contains('1')]

# String replacement
df['name_modified'] = df['name'].str.replace('User_', 'Employee_')

# Split strings
df['name_parts'] = df['name'].str.split('_')
df['prefix'] = df['name_parts'].str[0]
df['number'] = df['name_parts'].str[1]

print("String operations sample:")
print(df[['name', 'name_upper', 'user_number', 'name_modified']].head())

## 13. Performance Tips

In [ ]:
# Use vectorized operations instead of loops
# Good: df['new_col'] = df['col1'] * df['col2']
# Bad: df['new_col'] = df.apply(lambda x: x['col1'] * x['col2'], axis=1)

# Use categorical data for repeated strings
df['department_cat'] = df['department'].astype('category')
print(f"Memory usage - original: {df['department'].memory_usage(deep=True)} bytes")
print(f"Memory usage - categorical: {df['department_cat'].memory_usage(deep=True)} bytes")

# Use query() for complex filtering (can be faster)
# df.query('age > 30 and salary > 50000')

# Use loc/iloc for explicit indexing
# df.loc[df['age'] > 30, 'salary'] *= 1.1

# Chain operations efficiently
result = (df
          .query('age > 25')
          .groupby('department')['salary']
          .mean()
          .sort_values(ascending=False))

print("\nChained operation result:")
print(result)

## 14. Quick Data Exploration

In [ ]:
# Basic info
print("Dataset Info:")
print(f"Shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"Data types:\n{df.dtypes}")

# Summary statistics
print("\nSummary Statistics:")
print(df.describe())

# Value counts
print("\nDepartment distribution:")
print(df['department'].value_counts())

# Correlation matrix
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_cols].corr()
print("\nCorrelation matrix:")
print(correlation_matrix)

## Summary

This notebook covers the most essential pandas operations:

1. **Data Selection**: iloc, loc, boolean indexing
2. **Joining**: merge operations with different join types
3. **Duplicates**: detection and removal
4. **DateTime**: parsing, extraction, arithmetic
5. **Missing Values**: detection, filling, dropping
6. **Sampling**: random, stratified, top/bottom
7. **Grouping**: aggregation and transformation
8. **Data Transformation**: new columns, mapping, ranking
9. **Reshaping**: pivot tables, melting
10. **String Operations**: cleaning and extraction
11. **Performance**: memory optimization and efficient operations
12. **Exploration**: quick data overview methods

These operations cover 90% of typical data manipulation tasks in pandas.